### **This notebook aims at extracting the impervious information by using the pre-trained model ( <font color=red>data process is implemented in Google Colab</font>)**

In [1]:
## Mount on google drive
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### **We use Tensorflow 2.2.0 here!**

In [2]:
# !pip install tensorflow==2.2.0
import tensorflow as tf
print(tf.__version__)

2.2.0


In [3]:
import os
os.chdir("/content/drive/My Drive/Earth-Engine-with-Deep-Learning")
import matplotlib.pyplot as plt
import folium
import time
import json


In [65]:
## Super-parameter
# pre-trained model. Note: the pretrained model is trained with tensorflow 2.2.0.
Model_Drive_path = '/content/drive/My Drive/Earth-Engine-with-Deep-Learning/models/pretrain/unet_v1/model'
# result name
dataPred_prefix = 'Pred_Landsat8_2019'
## google drive 
data_folder = '/content/drive/My Drive/EE_Image'
result_drive_path = data_folder + '/' + dataPred_prefix + '.TFRecord'
## google cloud storage
Project = 'my-project-20200813'
Bucket = 'earth-engine-bucket-1'
result_gs_path = 'gs://' + Bucket + '/prediction_results' + '/' + dataPred_prefix + '.TFRecord'
## earth engine platform
ee_output_folder = 'users/xin_luo/Impervious_result_sz'
## Bands corresponding to the tfrecord file
Bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# data pipeline, the kernel shape and buffer should be consistency with the dataPred code
Kernel_shape = [256, 256]  
Kernel_buffer = [128, 128]

buffered_shape = [                # patch size with the buffered region
    Kernel_shape[0] + Kernel_buffer[0],
    Kernel_shape[1] + Kernel_buffer[1]]

### Automatically obtain the multiple tfrecord files (<font color=red> **the tfrecord files are obtained by running the code: dataPreparation/dataPred** </font>).

In [5]:
# Get a list of all the files in the tfrecord data folder.
os.chdir(data_folder)

filesList = !ls -1 
PredFilesList = [s for s in filesList if dataPred_prefix in s]
# Get the list of image files and the JSON mixer file.
tfrecordList = []
jsonFile = None
for f in PredFilesList:
    if f.endswith('.tfrecord.gz'):
        tfrecordList.append(f)
    elif f.endswith('.json'):
        jsonFile = f
tfrecordList.sort()
jsonFile_path = '/content/drive/My Drive/EE_Image/' +jsonFile

# Load the contents of the mixer file to a JSON object,and obtain the number of patches
jsonText = !cat {jsonFile}
mixer = json.loads(jsonText.nlstr)
patches = mixer['totalPatches']
mixer

{'patchDimensions': [256, 256],
 'patchesPerRow': 12,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
    0.0,
    113.75689836307785,
    0.0,
    -0.00026949458523585647,
    22.86472909516577]},
  'crs': 'EPSG:4326'},
 'totalPatches': 72}

### Parse the tfrecord data and read the tfrecord data into the tf.data.Dataset

In [6]:
imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
      for k in Bands
  ]
featuresDict = dict(zip(Bands, imageColumns))    # 用于tfrecord文件中数据描述的字典

def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, featuresDict)

def toTensor(inputs):
    inputsList = [inputs.get(key) for key in Bands]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

# Create a dataset from the TFRecord file(s) in Cloud Storage.
imageDataset = tf.data.TFRecordDataset(tfrecordList, compression_type='GZIP')
imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
imageDataset = imageDataset.map(toTensor).batch(1)
imageDataset

<BatchDataset shapes: (None, 384, 384, 6), types: tf.float32>

### Prediction: impervious surface mapping from satellite image

In [8]:
## load model
model_pretrain = tf.keras.models.load_model(Model_Drive_path)
# model_pretrain.summary()
predictions = model_pretrain.predict(imageDataset, steps=patches, verbose=1)

72/72 [==============================] - 2s 26ms/step


### <font color=red> **Write the predictions into a tfrecord file.** </font>
### ！Write the result to the google cloud storage, and then upload to earth engine (<font color=red>!awaiting to to</font>).

In [ ]:
## Authenticate to Google Earth engine
from google.colab import auth
auth.authenticate_user()
# ## Authenticate to Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
writer = tf.io.TFRecordWriter(result_gs_path)
i_patch = 0
for predPatch in predictions:
    print('Writing patch ' + str(i_patch) + '...')
    predPatch = predPatch[      # clip the overlay region
        Kernel_buffer[0]//2:Kernel_buffer[0]//2+Kernel_shape[0], 
        Kernel_buffer[1]//2:Kernel_buffer[1]//2+Kernel_shape[1]
        ]
    # Creat an example
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    i_patch += 1
writer.close()


### **Upload the result to the Earth Engine**

In [ ]:
# !pip install oauth2client==3.0.0

# 1. upload .json file from google drive to google storage
jsonFile_gs_path = 'gs://' + Bucket+ '/prediction_results/' + jsonFile
# !gsutil -m cp -r '/content/drive/My Drive/EE_Image/*.json' {jsonFile_gs_dir}
# 2. upload both the .TFRecord file and .json file from google storage to the earth engine.
ee_output_image = ee_output_folder + '/' + dataPred_prefix
!earthengine upload image --asset_id={ee_output_image} {result_gs_path} {jsonFile_gs_path}


### The impervious surface can be visuallized util the result are uploaded to the earth engine. 

In [76]:
out_image = ee.Image(ee_output_folder + '/' + dataPred_prefix)

mapid = out_image.getMapId({'min': 0, 'max': 1})
map = folium.Map(location=[22.65, 114.17])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted impervious',
  ).add_to(map)
map.add_child(folium.LayerControl())
map